In [1]:
from openai import OpenAI

In [ ]:
# Connection parameters (only has to be executed once during a session)

# Note: `localhost` is only valid as long as you are logged in on madagaskarweihe. 
# Otherwise, you will need to perform port forwarding on your machine using `ssh -fNL 8006:127.0.0.1:8006 ashousaa@madagaskarweihe`.
# This might require an ssh key to be uploaded to madagaskarweihe first.
VLLM_API_ENDPOINT = 'http://localhost:8006/v1' 
VLLM_KEY = 's7Vtzeyq3kfhVkPhlWdL95pPRBq36KDP1d5bBj54BqQ'
MODEL = "default"

In [ ]:
# Connection to LLM server (only has to be executed once during a session)
client = OpenAI(api_key=VLLM_KEY,
                base_url=VLLM_API_ENDPOINT)

In [ ]:

# Chunk documents, save results to a dictionary where each chunk has an id and save the chunks to a CSV file

import fitz  # Library for working with PDF files.
import csv # Library for working with CSV files.

def chunk_pdf_by_marker(pdf_path, marker="#"):
    doc = fitz.open(pdf_path)
    full_text = ""

    # Extract text from each page and concatenate
    for page in doc:
        full_text += page.get_text()
        # print(full_text)

    # Split by the marker
    chunks = [chunk.strip() for chunk in full_text.split(marker) if chunk.strip()] # Splits the text by the marker and iterates on each chunk and cleans the whitespace. Also, it removes empty strings after cleaning.

     # Store chunks in a dictionary
    chunk_dict = {f"chunk_{i+1}": chunk for i, chunk in enumerate(chunks)} # After the colon is the value of the dictionary, which is the chunk of text. The key is the chunk number, which is created by iterating over the chunks and adding 1 to the index.

    return chunk_dict


# Example usage
pdf_path = "Doc 4 flat.pdf"
chunk_dict = chunk_pdf_by_marker(pdf_path)

# Print the first 3 chunks entirely
print("--- First 3 Chunks ---")
for key, value in list(chunk_dict.items())[:4]:  # preview first 3 chunks
    print(f"{key}:\n{value}\n{'='*30}\n")

# Also save chunks with ids as a csv file
def save_chunks_to_csv(chunks, output_file="chunks.csv"):
    with open(output_file, mode="w", encoding="utf-8", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["chunk_id", "chunk_text"])  # Write header

        for i, chunk in enumerate(chunks):
            chunk_id = f"chunk_{i + 1}"
            writer.writerow([chunk_id, chunk])
            writer.writerow([]) # Empty row to create a line break

# Call the save_chunks_to_csv function with the values from the chunk_dict
save_chunks_to_csv(list(chunk_dict.values()))

In [ ]:

# Generate questions using the LLM and save to a text file named "response.txt"
with open("response.txt", "w") as f:
    for id in chunk_dict:
        doc = chunk_dict[id]
        # define prompts
        messages = [
            {"role": "system", "content": "You are about to go on a bussiness trip and want to ask very precise questions. Only output the question, no additional information."},
            # {"role": "user", "content": f"Imagine you are planning a business trip. What five questions would you ask after reading this document? {doc} Do not include the question if it is not stated in the chunk" },  
            # {"role": "user", "content": f"Generate a list of natural-sounding questions that a traveling employee would have. The questions should reflect a genuine need to understand this information for their reimbursement claim.Only generate questions for which the corresponding answer is explicitly present here.{doc}"} 
            {"role": "user", "content": f"What specific questions would I, as a traveling employee, ask that could be answered *solely by the information provided in this chunk*? Only generate questions for which the corresponding answer is explicitly present here.{doc}"}
            # {"role": "user", "content": f"As a traveling employee, what are some key, natural questions I would have to understand my rights, responsibilities, and potential reimbursements related to this specific information? Please generate questions that reflect a genuine need for clarity on how this impacts my expense report and reimbursement. Ensure the answer to each question is explicitly stated within this chunk, and format each question on a new line, numbered as in the examples you provided {doc}"}  
              ] 
        
        # send prompts and wait for answer
        response = client.chat.completions.create(
                    model=MODEL,
                    messages=messages,
                    seed=42,
                    temperature=0.7,
            )
      
        # Get and clean the response
        raw_response = response.choices[0].message.content.strip() 

        # Clean up the LLM's response by splitting into lines, removing leading/trailing noise, and filtering out empty lines
        questions = [q.strip("0123456789).:- ") for q in raw_response.split("\n") if q.strip()] 

        # Write each question on a new line with the chunk ID
        for question in questions:
            f.write(f"{question} || {id}\n") 

        # Add a blank line to separate questions from different chunks
        f.write("\n")

Code to iterate over models:

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed ,pipeline

# def load_model(model_name):
#     tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=MODEL_CACHE_DIR)
#     model = AutoModelForCausalLM.from_pretrained(
#         model_name,
#         torch_dtype="auto",
#         cache_dir=MODEL_CACHE_DIR
#     ).to("cuda:0")
#     return model, tokenizer

In [ ]:
# def generate(system_prompt: str, user_prompt: str, model, tokenizer, temperature: float = 0.7, seed: int = 42, enable_thinking=False) -> str:
#     set_seed(seed=seed)
#     messages = [
#         {"role": "system", "content": system_prompt},
#         {"role": "user", "content": user_prompt}
#     ]
#     text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=enable_thinking)
#     model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
#     generated_ids = model.generate(**model_inputs, max_new_tokens=32768)
#     output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
#     content = tokenizer.decode(output_ids, skip_special_tokens=True)
#     return content


In [ ]:
# import csv
# import gc
# import torch


# k = 10
# prompt_id = "v1"

# # Models to iterate over
# model_names = [
#     "Qwen/Qwen3-0.6B",
#     "Qwen/Qwen3-1.7B",
#     "Qwen/Qwen3-4B",
#     "Qwen/Qwen3-8B",
# ]

# # Load test set
# with open("modified_final_test_set.csv", "r", encoding="utf-8") as f:
#     reader = csv.DictReader(f)
#     full_data = list(reader)

# queries_chunk_map = [
#     {
#         "chunk_id": row["Chunk"],
#         "question": row["Question"],
#         "reference": row["Answer"]
#     }
#     for row in full_data
# ]

# # Output files
# output_txt_path = "RAG_Output_Answers_ALL_MODELS.txt"
# output_csv_path = "RAG_Output_Answers_ALL_MODELS.csv"

# with open(output_txt_path, "w", encoding="utf-8") as txt_file, \
#      open(output_csv_path, mode='w', newline='', encoding='utf-8') as csv_file:

#     fieldnames = [
#         "Model Used", "Question Index", "Question", "Original Chunk",
#         "Chunks Retrieved", "Answer", "Reference Answer", "Prompt Used"
#     ]
#     writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
#     writer.writeheader()

#     for model_name in model_names:
#         print(f"\n🔄 Loading model: {model_name}")
#         model, tokenizer = load_model(model_name)

#         correct_answers_count = 0

#         for i, item in enumerate(queries_chunk_map, start=1):
#             chunk_id = item["chunk_id"]
#             query = item["question"]
#             reference_answer = item["reference"]

#             top_k_chunk_ids = get_top_matching_chunks(query, k)
#             result = get_CSVcolumn('/mount/arbeitsdaten/studenten4/ashousaa/chunks.csv', 'chunk_text')
#             top_k_documents = [result[int(c_id.split("_")[1]) - 1] for c_id in top_k_chunk_ids]
#             top_k_string = "\n".join(top_k_documents)

#             prompt_text = f"Context: {top_k_string} \n Question: {query} \n Answer the question using only the context provided."

#             raw_response = generate(
#                 system_prompt="Please answer the user question in a faithful way.",
#                 user_prompt=prompt_text,
#                 model=model,
#                 tokenizer=tokenizer,
#                 temperature=0.7,
#                 seed=42
#             ).strip()

#             # Evaluate if correct chunk was found
#             normalized_chunk_id = normalize_string(chunk_id)
#             normalized_top_k_chunk_ids = [normalize_string(a) for a in top_k_chunk_ids]
#             if normalized_chunk_id in normalized_top_k_chunk_ids:
#                 correct_answers_count += 1

#             # Write to text file
#             txt_file.write(f"[{model_name}] Question {i} (Original Chunk ID: {chunk_id})\n")
#             txt_file.write(f"Question: {query}\n")
#             txt_file.write(f"Top-k Chunks Used: {top_k_chunk_ids}\n")
#             txt_file.write(f"Answer: {raw_response}\n\n")

#             # Write to CSV
#             writer.writerow({
#                 "Model Used": model_name,
#                 "Question Index": i,
#                 "Question": query,
#                 "Original Chunk": chunk_id,
#                 "Chunks Retrieved": "; ".join(top_k_chunk_ids),
#                 "Answer": raw_response,
#                 "Reference Answer": reference_answer,
#                 "Prompt Used": prompt_id
#             })

#         print(f"✅ Finished {model_name}: {correct_answers_count}/{len(queries_chunk_map)} correct chunks")

#         # 🧹 Free memory
#         del model
#         del tokenizer
#         gc.collect()
#         torch.cuda.empty_cache()


Code to Generate Model By Model

In [1]:
from common_methods import get_CSVcolumn, normalize_string, get_top_matching_chunks


/mount/arbeitsdaten/studenten4/ashousaa/.env/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os # Imports the os module, which provides a way to interact with the operating system.

MODEL_CACHE_DIR = '/mount/arbeitsdaten/asr-2/vaethdk/resources/weights/llm/students' # Tells the Hugging Face Transformers library to store downloaded model components in that specific directory.
os.environ['TRANSFORMERS_CACHE'] = MODEL_CACHE_DIR # Sets an environment variable that tells the transformers library to use the specified cache directory.


In [3]:
# Imports necessary classes from the transformers library which is a library that provides pre-trained models and tokenizers for natural language processing tasks.
# The following classes are imported:
# 1. AutoModelForCausalLM: A generic class that can automatically load pre-trained language models for text generation (causal language modeling).
# 2. AutoTokenizer: A generic class that can automatically load the appropriate tokenizer for a given pre-trained model. Tokenizers are used to convert text into numerical tokens that the model can understand and vice versa.
# 3. set_seed: A function to set the random seed for reproducibility of the generation process.
# 4. pipeline: A high-level API for using pre-trained models for various tasks, such as text generation, translation, etc.

from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed ,pipeline


# The model name points to a repository on huggingface, where it will load the model and configuration from, e.g.: https://huggingface.co/Qwen/Qwen3-0.6B

# model_name = "Qwen/Qwen3-0.6B"
# model_name = "Qwen/Qwen3-1.7B"
model_name = "Qwen/Qwen3-4B"
# model_name = "Qwen/Qwen3-8B"
# model_name = "Qwen/Qwen3-14B"


# model_name = "meta-llama/Llama-3.2-1B-Instruct"
# model_name = "meta-llama/Llama-3.2-3B-Instruct"

# model_name = "microsoft/Phi-3-mini-128k-instruct"
# model_name = "microsoft/Phi-3-small-128k-instruct"
# model_name = "microsoft/Phi-3-medium-128k-instruct"


# Load the tokenizer and the model (you can leave this block unchanged)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=MODEL_CACHE_DIR) #trust_remote_code=True) # Loads the tokenizer for the specified model from the cache directory.
model = AutoModelForCausalLM.from_pretrained(  # This is the key function from the transformers library that handles the loading process.
    model_name,
    torch_dtype="auto", # This argument specifies the data type of the model parameters. "auto" will automatically choose the best data type based on the hardware.
    cache_dir=MODEL_CACHE_DIR, #This optional argument specifies a local directory where the downloaded model files (configuration, weights, etc.) will be stored.
    # trust_remote_code=True, # This argument allows the model to use remote code for custom layers or configurations, which is necessary for some models that require specific implementations.
    
).to("cuda:0") # This is a PyTorch method that moves the entire loaded model (all its parameters and buffers) onto the first CUDA-enabled GPU available on your system

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 36.52it/s]


In [4]:
# This method is for:
# 1. converting the queries into the right template format
# 2. tokenizing the prompt text
# 3. converting the tokens to token ids
# 4. running the token ids through the model, generating output token ids
# 5. converting the output token ids back to text tokens


enable_thinking=False

def generate(system_prompt: str, user_prompt: str, temperature: float = 0.7, seed: int = 42, enable_thinking=enable_thinking) -> str:
    # set a random seed for reproducability (otherwise, calling generation twice can result in different texts)
    set_seed(seed=seed)

    # Convert the prompts into the correct template
    messages = [
        {"role": "system", "content": system_prompt}, 
        {"role": "user", "content": user_prompt}
    ]
    text = tokenizer.apply_chat_template( # Applies the specific chat template defined for the Qwen model to format the messages into a single text string that the model expects as input. 
        messages,
        tokenize=False,
        add_generation_prompt=True, # Adds a special token that signals the start of the generation.
        enable_thinking=enable_thinking # A parameter specific to some Qwen models to control a "thinking" step during generation.
    )
    # Convert prompt texts to tokens, move the token ids to the GPU
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # Passes the input tokens and set max new tokens number to finally generate the output token ids
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=32768
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

    # Convert the output token ids to text tokens
    content = tokenizer.decode(output_ids, skip_special_tokens=True)
    return content

In [ ]:
import csv

k = 10
correct_answers_count = 0

encoding_used="mpnet"
model_shortname = model_name.split("/")[-1]  # Remove slash inside the name cause gives error

# ======= SELECT PROMPT TO USE =======


# # --- Prompt 1: Basic RAG Prompt ---
prompt_id = "Basic_RAG_Prompt"
def build_prompt(context, query):
    return f"Context: {context} \n Question: {query} \n Answer the question using only the context provided."


# # # --- Prompt 2: Chain-of-Thought (CoT) ---
# prompt_id = "COT_Prompt"
# def build_prompt(context, query):
#     return (
#         f"Based on the retrieved context: {context}, "
#         f"answer the question {query} step by step, first identifying key facts, "
#         f"then reasoning through the answer."
#     )

# # # --- Prompt 3: No Context Provided ---
# prompt_id = "No_Context_Prompt"
# def build_prompt(context, query):
#     return f"Answer the query: {query} \n Answer:"

# =====================================

# === Load test set ===
# with open("modified_final_test_set.csv", "r", encoding="utf-8") as f:
#     reader = csv.DictReader(f)
#     full_data = list(reader)
with open("NEW_modified_final_test_set.csv", "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    full_data = list(reader)

queries_chunk_map = [ 
    {
        "chunk_id": row["Chunk"],
        "question": row["Question"],
        "reference": row["Answer"],
        "type": row["Type"],  
        "source_QID": row["Source_QID"]
    }
    
    for row in full_data
]

# === Output files ===

# # For trying different model sizes
# csv_path = f"NEW_RQ1_RAG_Answers_{prompt_id}_{model_shortname}.csv"

# For trying different generation prompts
# csv_path = f"NEW_RQ2_RAG_Answers_{prompt_id}_{model_shortname}.csv"

# For trying enabling thinking

# # Testing enabling thinking with different model sizes
# csv_path = f"NEW_RQ3_size_RAG_Answers_{prompt_id}_{model_shortname}.csv"

# # # Testing enabling thinking with different prompts
# csv_path = f"NEW_RQ3_prompt_RAG_Answers_{prompt_id}_{model_shortname}.csv"

with open(csv_path, mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = [
       "Generation Model", "Question Index", "Question", "Type", "Source_QID", "Original Chunk",
        "Chunks Retrieved", "Generated Answer", "Reference Answer", "Generation Prompt Used", "Encoding Used","Enable Thinking"
    ]
    
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames) 
    # Creates a CSV writer object that will write dictionaries to the CSV file. 
    # The keys of the dictionaries are the fieldnames which are column headers in the CSV file.
    # The values of the dictionaries are the data that will be written to the CSV file, specified when we call writer.writerow().

    writer.writeheader() # Actually writes the header row to the CSV file.

    for i, item in enumerate(queries_chunk_map, start=1):
        chunk_id = item["chunk_id"]
        query = item["question"]
        reference_answer = item["reference"]

        # Get ids of top-k chunks
        top_k_chunk_ids = get_top_matching_chunks(query, k)

        # Load corressponding chunk texts
        # csv_file_path = '/mount/arbeitsdaten/studenten4/ashousaa/chunks.csv'
        csv_file_path = '/mount/arbeitsdaten/studenten4/ashousaa/NEW_chunks.csv'
        result = get_CSVcolumn(csv_file_path, 'chunk_text')
        # The following line, takes each chunk ID from the top_k_chunk_ids, extracts the numeric part of it and converts to it , and substracts one to be 0-based index. Then it retrieves the text from 'result'.
        top_k_documents = [result[int(c_id.split("_")[1]) - 1] for c_id in top_k_chunk_ids] 
    

        # Build prompt
        top_k_string = "\n".join(top_k_documents)
        context = top_k_string if "No_Context" not in prompt_id else ""
        print (f"Context: {context}")  # Debugging line to see the context being used
        prompt_text = build_prompt(context, query)
        # print(f"Prompt text: {prompt_text}")  # Debugging line to see the prompt text

        # Generate response
        raw_response = generate(
            system_prompt="Please answer the user question in a faithful way.",
            user_prompt=prompt_text,
            temperature=0.7,
            seed=42
        ).strip()

        # Correct chunk detection
        normalized_chunk_id = normalize_string(chunk_id)
        normalized_top_k_chunk_ids = [normalize_string(a) for a in top_k_chunk_ids]
        correct_chunk_found = normalized_chunk_id in normalized_top_k_chunk_ids
        if correct_chunk_found:
            correct_answers_count += 1


        # Write to CSV
        writer.writerow({
            "Generation Model": model_name,
            "Question Index": i,
            "Question": query,
            "Type": item["type"],
            "Source_QID": item["source_QID"],
            "Original Chunk": chunk_id,
            "Chunks Retrieved": "; ".join(top_k_chunk_ids),
            "Generated Answer": raw_response,
            "Reference Answer": reference_answer,
            "Generation Prompt Used": prompt_id,
            "Encoding Used": encoding_used,
            "Enable Thinking": enable_thinking
               
        })

# Print the number of correct answers
# print(f"✅ Finished {model_name}: {correct_answers_count})")
print(f"✅ Finished {prompt_id}: {correct_answers_count})")


Context: State Travel Expense Act 
(LRKG) §  Official Trips and Errands () Official trips in the sense of this Act are journeys 
undertaken to conduct official business outside the usual place of work, which have been 
ordered or approved by the responsible superior, unless an order or approval is not applicable 
due to the nature of the official’s office or the nature of the official business. The order or 
approval must be given in writing or electronically. Official trips also include journeys from a 
location serving as a temporary residence to the place of work, provided that the conditions of 
sentences 1 and 2 are otherwise met. Official trips should only be carried out if a less costly 
method of conducting the official business is not possible or reasonable.
State Travel Expense 
Act (LRKG) §  Official Trips and Errands () Official errands are journeys undertaken to conduct 
official business at the place of work or residence outside the official premises, which have been 
ord

KeyboardInterrupt: 